In [1]:
!pip install transformers

     |████████████████████████████████| 2.2 MB 4.0 MB/s eta 0:00:01
     |████████████████████████████████| 3.3 MB 18.0 MB/s eta 0:00:01
     |████████████████████████████████| 733 kB 32.7 MB/s eta 0:00:01
     |████████████████████████████████| 895 kB 28.0 MB/s eta 0:00:01


In [23]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing
import transformers
import numpy as np
import os
import time

In [4]:
DATA_TRAIN = '../data/medical-text/train.dat'
DATA_TEST = '../data/medical-text/test.dat'

In [5]:
comments = open(DATA_TRAIN, 'rb').readlines()

In [6]:
def split_tab(comment):
    return comment.decode(encoding='utf-8').split('\t')[1]

In [7]:
comments_clean = list(map(split_tab, comments))

In [8]:
comments_clean = ' '.join(comments_clean)

In [9]:
print(f'Length of text: {len(comments_clean)} characters')

Length of text: 17797244 characters


In [10]:
vocab = sorted(set(comments_clean))
print(f'{len(vocab)} unique characters')

85 unique characters


In [11]:
vocab

['\n',
 ' ',
 '!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '=',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '[',
 ']',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [12]:
ids_from_chars = preprocessing.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [13]:
chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [14]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [15]:
all_ids = ids_from_chars(tf.strings.unicode_split(comments_clean, 'UTF-8'))
all_ids

<tf.Tensor: shape=(17797244,), dtype=int64, numpy=array([34, 60, 79, ..., 16,  2,  1])>

In [17]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
ids_dataset

<TensorSliceDataset shapes: (), types: tf.int64>

In [18]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

C
a
t
h
e
t
e
r
i
z


In [19]:
seq_length = 200
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

In [20]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [21]:
dataset = sequences.map(split_input_target)

In [22]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print()
    print("Target:", text_from_ids(target_example).numpy())

Input : b'Catheterization laboratory events and hospital outcome with direct angioplasty for acute myocardial infarction To assess the safety of direct infarct angioplasty without antecedent thrombolytic therap'

Target: b'atheterization laboratory events and hospital outcome with direct angioplasty for acute myocardial infarction To assess the safety of direct infarct angioplasty without antecedent thrombolytic therapy'


In [29]:
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

dataset

<PrefetchDataset shapes: ((64, 200), (64, 200)), types: (tf.int64, tf.int64)>

In [1]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))

NameError: name 'tf' is not defined

In [26]:
from transformers import BertGenerationDecoder

In [25]:
BertGenerationDecoder.

ImportError: 
GPT2Model requires the PyTorch library but it was not found in your environment. Checkout the instructions on the
installation page: https://pytorch.org/get-started/locally/ and follow the ones that match your environment.
